In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os

def make_catndog_dataframe():
    paths=[]
    dataset_gubuns=[]
    label_gubuns=[]
    
    for dirname, _, filenames in os.walk('/kaggle/input/cat-and-dog'):
        for filename in filenames:
            if '.jpg' in filename:
                file_path = dirname +'/'+filename
                paths.append(file_path)
                if '/training_set/' in file_path:
                    dataset_gubuns.append('train')  
                elif '/test_set/' in file_path:
                    dataset_gubuns.append('test')
                else: dataset_gubuns.append('N/A')

                # 파일의 절대 경로에 dogs가 있을 경우 해당 파일은 dog 이미지 파일이고, cats일 경우는 cat 이미지 파일임. 
                if 'dogs' in file_path:
                    label_gubuns.append('DOG')
                elif 'cats' in file_path:
                    label_gubuns.append('CAT')
                else: label_gubuns.append('N/A')
    data_df = pd.DataFrame({'path':paths, 'dataset':dataset_gubuns, 'label':label_gubuns})
    return data_df

In [ ]:
pd.set_option('display.max_colwidth', 200)
data_df = make_catndog_dataframe()
print('data_df shape:', data_df.shape)
data_df.head()

In [ ]:
from tensorflow.keras.utils import Sequence
import sklearn
import cv2

BATCH_SIZE=64
IMAGE_SIZE=160

class CnD_Dataset(Sequence):
    def __init__(self, image_filenames, labels, batch_size=BATCH_SIZE,augmentor=None, pre_func = None,shuffle=False):
        self.image_filenames= image_filenames
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = pre_func
        self.shuffle = shuffle
        
        if self.shuffle:
            self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.labels)/BATCH_SIZE))
    
    def __getitem__(self, index):
        image_name_batch = self.image_filenames[index*self.batch_size:(index+1)*self.batch_size]
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
        
        image_batch = np.zeros((image_name_batch.shape[0],IMAGE_SIZE,IMAGE_SIZE,3))
        
        for image_index in range(image_name_batch.shape[0]):
            image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]),cv2.COLOR_BGR2RGB)
            image = cv2.resize(image,(IMAGE_SIZE,IMAGE_SIZE))
            
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
            if self.pre_func is not None:
                image = self.pre_func(image)
            
            image_batch[image_index] = image
        return image_batch, label_batch
    
    def on_epoch_end(self):
        if(self.shuffle):
            self.image_filenames, self.labels = sklearn.utils.shuffle(self.image_filenames,self.labels)
        else:
            pass

In [ ]:
from sklearn.model_selection import train_test_split

def get_train_valid_test(data_df):
    train_df = data_df[data_df['dataset']=='train']
    test_df = data_df[data_df['dataset']=='test']
    
    train_path = train_df['path'].values
    train_label = pd.factorize(train_df['label'])[0]
    
    test_path = test_df['path'].values
    test_label = pd.factorize(test_df['label'])[0]
    
    tr_path, val_path, tr_label, val_label = train_test_split(train_path, train_label, test_size=0.5, random_state=2021 )
    
    return tr_path, tr_label, val_path, val_label, test_path, test_label

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import Xception, MobileNetV2

def create_model(model_name= 'mobilenet', verbose=False):
    input_tensor = Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3))
    if model_name == 'vgg16':
        base_model = VGG16(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'resnet50':
        base_model = ResNet50V2(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'xception':
        base_model = Xception(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'mobilenet':
        base_model = MobileNetV2(input_tensor=input_tensor, include_top=False,weights='imagenet')
        
    bm_output = base_model.output
    
    x = GlobalAveragePooling2D()(bm_output)
    if model_name != 'vgg16':
        x = Dropout(rate=0.5)(x)
    x = Dense(50, activation='relu', name='fc1')(x)
    output=Dense(1, activation='sigmoid',name='output')(x)
    
    model = Model(inputs = input_tensor, outputs=output)
    
    if verbose:
        model.summary()
    
    return model

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input as mobile_preprocess_input

def train_model(data_df, model_name, augmentor, preprocessing_func):
    tr_path, tr_label, val_path, val_label, test_path, test_label = get_train_valid_test(data_df)
    
    tr_ds = CnD_Dataset(tr_path, tr_label, batch_size=BATCH_SIZE, augmentor = augmentor,shuffle=True,pre_func = preprocessing_func)
    val_ds = CnD_Dataset(val_path, val_label, batch_size=BATCH_SIZE,augmentor = None, shuffle=False,pre_func = preprocessing_func)
    
    model = create_model(model_name = model_name)
    model.compile(optimizer=Adam(0.0001),loss='binary_crossentropy', metrics=['accuracy'])
    
    N_EPOCHS=20
    
    history = model.fit(tr_ds,epochs=N_EPOCHS, steps_per_epoch = int(np.ceil(tr_path.shape[0]/BATCH_SIZE)),
                       validation_data = val_ds, validation_steps = int(np.ceil(val_path.shape[0]/BATCH_SIZE)),verbose=1)
    return model, history

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input as mobile_preprocess_input
import tensorflow as tf

input_df, _ = train_test_split(data_df, test_size=0.7, random_state=2021)
mobil_model, mobile_history = train_model(input_df, 'mobilenet',None, mobile_preprocess_input)

In [ ]:
test_df = data_df[data_df['dataset']=='test']

test_path = test_df['path'].values
test_label = pd.factorize(test_df['label'])[0]

test_ds = CnD_Dataset(test_path, test_label, batch_size=BATCH_SIZE,augmentor = None, shuffle=False, pre_func=mobile_preprocess_input)
mobil_model.evaluate(test_ds)

In [ ]:
model = create_model(model_name='mobilenet')
model.summary()


In [ ]:
model.layers

In [ ]:
model.layers[-4:]

In [ ]:
from tensorflow.keras import layers

def train_model_fine_tune(data_df,model_name, augmentor, preprocessing_func):
    tr_path, tr_label, val_path, val_label, test_path, test_label = get_train_valid_test(data_df)
    
    tr_ds = CnD_Dataset(tr_path, tr_label, batch_size=BATCH_SIZE, augmentor=augmentor, 
                          shuffle=True, pre_func=preprocessing_func)
    val_ds = CnD_Dataset(val_path, val_label, batch_size=BATCH_SIZE, augmentor=None, 
                           shuffle=False, pre_func=preprocessing_func)
    
    model = create_model(model_name = model_name)
    model.compile(optimizer=Adam(0.0001),loss='binary_crossentropy',metrics=['accuracy'])
    
    for layer in model.layers[:-4]:
        layer.trainable = False
        
       
    FIRST_EPOCHS = 10
    SECOND_EPOCHS = 10
    
    history = model.fit(tr_ds, epochs=FIRST_EPOCHS, validation_data=val_ds, verbose=1)
    
    for layer in model.layers:
        if not isinstance(layer, layer.BatchNormalization):
            layer.trainable=True
    
    model.compile(optimizer=Adam(0.00001), loss='binary_crossentropy', metrics=['accuracy'])    
    history = model.fit(tr_ds, epochs=SECOND_EPOCHS, steps_per_epoch=int(np.ceil(tr_path.shape[0]/BATCH_SIZE)), 
                       validation_data=val_ds, validation_steps=int(np.ceil(val_path.shape[0]/BATCH_SIZE)),
                       verbose=1)
    
    return model, history

In [ ]:
mobile_model_tuned, mobile_tuned_history = train_model_fine_tune(input_df,'mobilenet',None, mobile_preprocess_input)

In [ ]:
mobile_model_tuned.evaluate(test_ds)